In [ ]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pickle
import os
from dotenv import load_dotenv

### FEtch Data from API

In [5]:
def get_alpha_vantage_news(api_key, ticker, topic=None):
    """
    Fetches news articles about `ticker` from the Alpha Vantage News API.
    Optionally filters by `topic` (e.g., "technology", "finance", etc.).
    Returns a list of dicts with article info (title, summary, sentiment, etc.).
    """

    # Alpha Vantage 'NEWS_SENTIMENT' endpoint
    url = "https://www.alphavantage.co/query"

    # Base params
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "apikey": api_key,
        "sort": "RELEVANCE"
    }

    # If a topic was provided, include it in the params
    # NOTE: The Alpha Vantage parameter for filtering by topic is "topics".
    # E.g., topics=technology,ipo
    if topic:
        params["topics"] = topic

    response = requests.get(url, params=params)
    print("Request URL:", response.url)  # Debugging: see the final request
    data = response.json()

    # Check for errors
    if "feed" not in data:
        print(f"Error fetching news from Alpha Vantage: {data}")
        return []

    articles = data["feed"]
    return articles

In [6]:
def get_company_overview(api_key, ticker):
    """
    Fetches the company overview from Alpha Vantage using function=OVERVIEW.
    Returns a Pandas DataFrame.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": ticker,
        "apikey": api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "Symbol" not in data:
        print(f"Error fetching company overview: {data}")
        return None

    # Convert JSON dictionary to a Pandas DataFrame
    overview_df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])
    overview_df.reset_index(inplace=True)
    overview_df.columns = ['Field', 'Value']

    return overview_df

In [7]:
load_dotenv()
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

In [8]:
articles = get_alpha_vantage_news(ALPHA_VANTAGE_API_KEY, "TSLA")

Request URL: https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=TSLA&apikey=4RZANL9BTFIUH6KF&sort=RELEVANCE


In [14]:
overview_data = get_company_overview(ALPHA_VANTAGE_API_KEY, "TSLA")

### Unstructured Loader

In [9]:
from langchain.document_loaders import UnstructuredURLLoader

In [10]:
def parse_article(articles):
    """
    Parses the article and returns the article text.
    """

    urls = []
    for i in range(len(articles)):
        urls.append(articles[i]['url'])

    loader = UnstructuredURLLoader(urls=urls)
    parsed_docs = loader.load()

    return parsed_docs

In [11]:
docs = parse_article(articles)

#### RecursiveTextSplitter

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
def split_text(docs):
    """
    Splits the text into smaller chunks.
    """

    text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', '.', ','],
    chunk_size=1000
    )

    chunks = text_splitter.split_documents(docs)
    texts = [chunk.page_content for chunk in chunks]

    return texts

In [16]:
chunks = split_text(docs)

In [17]:
len(chunks)

188

### FAISS Database

In [19]:
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
def buildIndex(texts):
    """
    Build a FAISS index from a list of text documents.
    Each document is encoded into a vector using SentenceTransformer.
    The vectors are then normalized and added to the FAISS index.
    Args:
        text (list): List of text documents to be indexed.
    Returns:
        index (faiss.Index): A FAISS index containing the normalized vectors.
    """
    
    encoder = SentenceTransformer("BAAI/bge-base-en")
    vectors = encoder.encode(texts)
    faiss.normalize_L2(vectors)
    
    dim = vectors.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(vectors)

    return index


In [21]:
index = buildIndex(chunks)

In [22]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x29e41b810> >

### Retrieval

In [67]:
def retrieval_faiss(query, index, k=10):
    """
    Perform a nearest neighbor search using a FAISS index.
    Args:
        query (str): The user query text.
        index: The FAISS index.
        k (int): Number of nearest neighbors to return.
    Returns:
        indi (list): List of tuples (index, score) for the k nearest neighbors.
    """
    
    encoder = SentenceTransformer("BAAI/bge-base-en")
    vec = encoder.encode(search_query)

    ##covert to same dim as source DB
    svec = np.array(vec).reshape(1,-1) 

    faiss.normalize_L2(svec)
    distances, indices = index.search(svec, k)
    
    return indices

In [64]:
search_query = "What's Tesla's latest model and price?"

In [68]:
retrieval_faiss(search_query, index, k=10)

array([[66, 47, 31, 16, 32, 35, 20, 25, 58, 39]])

In [42]:
for i in Indices.tolist()[0]:
    print(f"***********This is chunk: {i}", chunks[i])

***********This is chunk: 66 ▲ S&P 500 +---% | ▲ Stock Advisor +---% Join The Motley Fool

Accessibility Log In Help

Accessibility Menu

Free Article

You're reading a free article with opinions that may differ from The Motley Fool's Premium Investing Services. Become a Motley Fool member today to get instant access to our top analyst recommendations, in-depth research, investing resources, and more. Learn More

What's Going on With Tesla Stock?

By Parkev Tatevosian, CFA – Oct 29, 2024 at 5:05AM

NASDAQ: TSLA

Tesla

Market Cap

$898B

Today's Change

(-6.23%) -$17.39

Current Price

$261.71

Price as of March 6, 2025, 2:04 p.m. ET

A fantastic quarter has Tesla stock investors excited about the future.

Tesla (TSLA -6.23%) reported fantastic quarterly financial results, and the stock price is soaring in response.

Stock prices used were the afternoon prices of Oct. 24, 2024. The video was published on Oct. 26, 2024.
***********This is chunk: 47 ▲ S&P 500 +---% | ▲ Stock Advisor +---

### LangChain Retrieval (Ignore)

In [58]:
from langchain.chains import RetrievalQAWithSourcesChain

In [67]:
from langchain import OpenAI

In [61]:
LLM_API_KEY = os.getenv("OPEN_ROUTER_KEY")

In [68]:
llm = OpenAI(temperature=0.9, max_tokens=500)

/var/folders/jj/1w9v7wf56_ldb_sbfv129l280000gn/T/ipykernel_80058/2582954192.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9, max_tokens=500)


ValidationError: 1 validation error for OpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0.9, 'max...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [65]:
llm = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=LLM_API_KEY
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [59]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=index.as_retriever())

NameError: name 'llm' is not defined